In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,2021-09-26 02:35:25,42.6001,-73.9662,60.58,89,100,2.48
1,1,Synya,RU,2021-09-26 02:39:43,65.3718,58.0387,36.21,80,100,3.33
2,2,Shizukuishi,JP,2021-09-26 02:39:43,39.6833,140.9833,68.77,78,100,6.04
3,3,Longyearbyen,SJ,2021-09-26 02:35:26,78.2186,15.6401,39.04,87,75,6.91
4,4,Kodinar,IN,2021-09-26 02:39:44,20.7903,70.7031,80.33,80,79,7.14


In [3]:
city_data_df.dtypes


City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat","Lng"]]

# 2. Get the Max temp data
max_temp = city_data_df["Max Temp"]
#Get the maximum temp 
temps = []
for temp in max_temp:
    temps.append(max(temp,0))

# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
#heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp)
#heatmap_layer = gmaps.heatmap_layer(locations, weights=temps)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                dissipating=False, max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)

# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Heat map for humidity 
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#Heat map for cloudiness 
locations = city_data_df[["Lat","Lng"]]
cloudiness = city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness,
                                dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Heat map for Wind Speed 
locations = city_data_df[["Lat","Lng"]]
wind_speed = city_data_df["Wind Speed"]

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed,
                                dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Kodinar,IN,2021-09-26 02:39:44,20.7903,70.7031,80.33,80,79,7.14
11,11,Tuban,ID,2021-09-26 02:39:46,-6.8976,112.0649,86.36,64,95,5.79
12,12,Mareeba,AU,2021-09-26 02:36:52,-17.0000,145.4333,86.43,49,90,7.52
13,13,Berbera,SO,2021-09-26 02:39:46,10.4396,45.0143,87.98,58,37,3.69
15,15,Sinnamary,GF,2021-09-26 02:37:58,5.3833,-52.9500,79.45,86,30,9.10
17,17,Hilo,US,2021-09-26 02:35:13,19.7297,-155.0900,82.13,83,75,11.50
27,27,Marzuq,YE,2021-09-26 02:37:17,14.4000,46.4667,75.06,35,57,2.39
39,39,Maceio,BR,2021-09-26 02:40:13,-9.6658,-35.7353,80.04,89,75,9.22
41,41,Portland,US,2021-09-26 02:32:56,45.5234,-122.6762,77.59,54,1,1.01
42,42,Tevaitoa,PF,2021-09-26 02:40:14,-16.7833,-151.5000,79.27,72,0,7.40


In [10]:
preferred_cities_df.notnull().sum()

City_ID       174
City          174
Country       174
Date          174
Lat           174
Lng           174
Max Temp      174
Humidity      174
Cloudiness    174
Wind Speed    174
dtype: int64

In [18]:
# Create  hotel_df to store hotel names along with city, country, max temp, and coordinates.
#copy is from preffered cities
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Kodinar,IN,80.33,20.7903,70.7031,
11,Tuban,ID,86.36,-6.8976,112.0649,
12,Mareeba,AU,86.43,-17.0000,145.4333,
13,Berbera,SO,87.98,10.4396,45.0143,
15,Sinnamary,GF,79.45,5.3833,-52.9500,
17,Hilo,US,82.13,19.7297,-155.0900,
27,Marzuq,YE,75.06,14.4000,46.4667,
39,Maceio,BR,80.04,-9.6658,-35.7353,
41,Portland,US,77.59,45.5234,-122.6762,
42,Tevaitoa,PF,79.27,-16.7833,-151.5000,


In [22]:
#Example_
# Set parameters to search for a hotel.
#params = {
#    "key":g_key,
#    "radius": 5000,
#    "type": "lodging",
#    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
#hotels = requests.get(base_url, params=params).json()

#hotels
#Number of hotels in
#len(hotels["results"])

In [23]:
# Iterate through the DataFrame.
params = {
    "key":g_key,
    "radius": 5000,
    "type": "lodging"}

#Start DATA research
print("------------------------------------")
print("Start hotel analysis")
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
     # Add the latitude and longitude to location key for the params dictionary.
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping")

print("End of hotel research")

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [24]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Kodinar,IN,80.33,20.7903,70.7031,residency of N.B vadhel
11,Tuban,ID,86.36,-6.8976,112.0649,Hotel Mustika
12,Mareeba,AU,86.43,-17.0000,145.4333,Trinity Plains Tourist Park
13,Berbera,SO,87.98,10.4396,45.0143,IBCO REAL ESTATE PROPERTY
15,Sinnamary,GF,79.45,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
...,...,...,...,...,...,...
541,Duliagaon,IN,84.07,27.3667,95.3167,Kanchanjongha Complex
542,Vieux-Habitants,GP,76.57,16.0589,-61.7659,Gîtes Dardanelle
544,Cockburn Town,TC,82.92,21.4612,-71.1419,Osprey Beach Hotel
552,Bangkalan,ID,85.95,-7.0455,112.7351,SPOT ON 2591 Griya Demangan Syariah


In [28]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info


['\n<dl>\n<dt>Hotel Name</dt><dd>residency of N.B vadhel</dd>\n<dt>City</dt><dd>Kodinar</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Max Temp</dt><dd>80.33 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Mustika</dd>\n<dt>City</dt><dd>Tuban</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Max Temp</dt><dd>86.36 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Trinity Plains Tourist Park</dd>\n<dt>City</dt><dd>Mareeba</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Max Temp</dt><dd>86.43 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>IBCO REAL ESTATE PROPERTY</dd>\n<dt>City</dt><dd>Berbera</dd>\n<dt>Country</dt><dd>SO</dd>\n<dt>Max Temp</dt><dd>87.98 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>ALETHEIA Appartement Sinnamary</dd>\n<dt>City</dt><dd>Sinnamary</dd>\n<dt>Country</dt><dd>GF</dd>\n<dt>Max Temp</dt><dd>79.45 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hilo Hawaiian Hotel</dd>\n<dt>City</dt><dd>Hilo</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>82.13 °F</dd>\n

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))